In [ ]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/x_y_dataset.csv")

In [ ]:
df = df.drop(df.index[0:22])
df.head(24)


,lemmatized,Etiket
22,"['da', 'orospu', 'cocuklari', 'hep', 'diye', '...",nefret
23,"['ciddi', 'ol', 'tüm', 'dünya', 'beyn', 'bil',...",saldırgan
24,"['kayıt', 'istihdam', 'geniş', 'program', 'gör...",hiçbiri
25,"['hastane', 'ait', 'suriye', 'ptt', 've', 'ait...",nefret
26,"['köle', 'git', 'suriye', 'gibi', 'bugün']",hiçbiri
27,"['çocuk', 'sadece', 'türkiye', 'değil', 'ırak'...",hiçbiri
28,"['suriye', 'ol', 'bil', 'httpstcodknrytjfxt']",hiçbiri
29,"['keşke', 'sadece', 'suriye', 'çalış', 'işvere...",hiçbiri
30,"['izin', 'suriye', 'afgan', 'pakistan', 'polit...",hiçbiri
31,"['ülke', 'büyükelçi', 'dahil', 'suriye', 'iste']",nefret


In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [ ]:

# Metinleri kelime listelerine bölme
sentences = [text.split() for text in df['lemmatized']]

# Word2Vec modelini tanımlama ve eğitme
word2vec_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2, workers=4, sg=1)
word2vec_model.train(sentences, total_examples=len(sentences), epochs=10)

# Kelime vektörlerini alma
word_vectors = word2vec_model.wv
vector_size = 100

# Metinleri Word2Vec vektörlerine dönüştürme
def vectorize_text(text, model, vector_size):
    words = text.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

# Tüm metinleri vektörleştirme
X = np.array([vectorize_text(text, word_vectors, vector_size) for text in df['lemmatized']])


In [ ]:
# Hedef değişkeni (y) sayısal değerlere dönüştürme
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['Etiket'])

# Sınıf dönüşüm haritasını yazdırma
print("Class mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


Class mapping: {'hiçbiri': 0, 'nefret': 1, 'saldırgan': 2}


In [ ]:
# Eğitim ve test verilerini ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# SMOTE ile veri dengeleme
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train, y_train)

# SMOTETomek ile veri dengeleme
smote_tomek = SMOTETomek(random_state=42)
X_smotetomek, y_smotetomek = smote_tomek.fit_resample(X_train, y_train)

# RandomUnderSampler ile veri dengeleme
undersampler = RandomUnderSampler(random_state=42)
X_undersampled, y_undersampled = undersampler.fit_resample(X_train, y_train)

# Resampling yöntemlerini listeleme
resampling_methods = {
    "Original": (X_train, y_train),
    "SMOTE": (X_smote, y_smote),
    "SMOTETomek": (X_smotetomek, y_smotetomek),
    "RandomUnderSampler": (X_undersampled, y_undersampled)
}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validatio

In [ ]:
# Modelleri tanımlama
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    "Artificial Neural Network (ANN)": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)
}


In [ ]:
# Performans sonuçları için tablo oluşturma
results = []

for method_name, (X_resampled, y_resampled) in resampling_methods.items():
    for model_name, model in models.items():
        # Modeli eğitme
        model.fit(X_resampled, y_resampled)

        # Test verilerinde tahmin
        y_pred = model.predict(X_test)

        # Performans metriklerini hesaplama
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        # Sonuçları tabloya ekleme
        results.append({
            "Resampling Method": method_name,
            "Model": model_name,
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1
        })

# Sonuçları DataFrame olarak gösterme
results_df = pd.DataFrame(results)
print(results_df)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:37:21] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 8179, number of used features: 100
[LightGBM] [Info] Start training from score -0.280737
[LightGBM] [Info] Start training from score -1.476166
[LightGBM] [Info] Start training from score -4.118976


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:38:16] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007836 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 18531, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:39:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [15:40:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000956 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13371
[LightGBM] [Info] Number of data points in the train set: 399, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


     Resampling Method                            Model  Accuracy  Precision  \
0             Original              Logistic Regression  0.804890   0.776845   
1             Original                    Random Forest  0.807335   0.795155   
2             Original                          XGBoost  0.811247   0.785236   
3             Original                         LightGBM  0.819071   0.795179   
4             Original  Artificial Neural Network (ANN)  0.791198   0.786367   
5                SMOTE              Logistic Regression  0.653790   0.797481   
6                SMOTE                    Random Forest  0.808802   0.785566   
7                SMOTE                          XGBoost  0.805868   0.791977   
8                SMOTE                         LightGBM  0.807824   0.803473   
9                SMOTE  Artificial Neural Network (ANN)  0.793643   0.790156   
10          SMOTETomek              Logistic Regression  0.652323   0.796773   
11          SMOTETomek                  

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
results_df


,Resampling Method,Model,Accuracy,Precision,Recall,F1-Score
0,Original,Logistic Regression,0.804890,0.776845,0.804890,0.779132
1,Original,Random Forest,0.807335,0.795155,0.807335,0.764735
2,Original,XGBoost,0.811247,0.785236,0.811247,0.788098
3,Original,LightGBM,0.819071,0.795179,0.819071,0.795497
4,Original,Artificial Neural Network (ANN),0.791198,0.786367,0.791198,0.788448
5,SMOTE,Logistic Regression,0.653790,0.797481,0.653790,0.709611
6,SMOTE,Random Forest,0.808802,0.785566,0.808802,0.794096
7,SMOTE,XGBoost,0.805868,0.791977,0.805868,0.797859
8,SMOTE,LightGBM,0.807824,0.803473,0.807824,0.805342
9,SMOTE,Artificial Neural Network (ANN),0.793643,0.790156,0.793643,0.791806
